# Back to Basics: Let's Build a Language Model – Andrej Jovanović

![title](images/llm.jpg)

Large language models have taken the world by storm. Anyone heard of ChatGPT or something like that? No? Me neither…

The powerful thing about these technologies are using neural networks to create successful models of language. However, language model machinery was not always so grand and complex: we once relied on basic probability theory. I find it helpful to view these language models through this probabilistic lens: it helps to understand and normalise them, in a sense. Join me in this tutorial where we create a relatively impoverished n-gram language model from scratch!

*Note: this Jupyter Notebook ties in with a set of lecture slides that explain the theoretical backbone of the code. The link for which can be found here : This tutorial is based of a few resources which can be found here: \
https://www.geeksforgeeks.org/n-gram-language-modelling-with-nltk/ \
https://web.stanford.edu/~jurafsky/slp3/3.pdf \
https://github.com/nltk/nltk/blob/develop/nltk/lm/__init__.py*

## Step 1: What is our data?
For this tutorial, we are going to use the famous Gutenberg corpus. We will use the NLTK package for this - it is also very useful for a wide variety of NLP tasks.

In [ ]:
# Collect all of our imports at the top of the notebook - just for convenience ;)
import os, nltk, string, random
from collections import Counter, defaultdict
import nltk
nltk.download()
from nltk import FreqDist, word_tokenize, sent_tokenize 
from nltk.corpus import gutenberg, stopwords, webtext
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams, pad_sequence, bigrams, everygrams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
sentence_data = gutenberg.sents()
sentence_data[480]

As you can see, we are returned a list of lists. Where each list within the list is a sentence. These sentences actually come from novels - this sentence in particular comes from Emma by Jane Austen. Our goal now is to create a model of the English language, based on novel contained in the Gutenberg corpus. However, we first need to clean the data before we are ready to create our n-gram language model.

## Step 2: Cleaning the data

First, we need to remove some stop words. These are words that are very frequently occuring in English and will distract from our language modelling task - bloating some of the probabilities.

In [ ]:
stop_words = set(stopwords.words('english'))
string.punctuation = string.punctuation +'"'+'"'+'-'+'''+'''+'—'
string.punctuation
removal_list = list(stop_words) + list(string.punctuation)+ ['lt','rt']  
def remove_stopwords(x):     
    y = []
    for pair in x:
        count = 0
        for word in pair:
            if word in removal_list:
                count = count or 0
            else:
                count = count or 1
        if (count==1):
            y.append(pair)
    return (y)

Now, we want to build up our vocabulary. To achieve this, we need to first do a number of things \
- To start with, we need to tokenise our text. In this context, this means cleaning our input data so that all the words are lowercase.
- Then, we want to create our  unigram, bigram and trigram vocabularies. 


In [ ]:
# generate unigrams bigrams trigrams
SOS = "<s>"
EOS = "</s>"
unigram=[]
bigram=[]
trigram=[]
tokenized_text=[]
for sentence in sentence_data:
  sentence = list(map(lambda x:x.lower(),sentence))
  for word in sentence:
        if word== '.':
            sentence.remove(word) 
        else:
            unigram.append(word)
  tokenized_text.append(sentence)
  bigram.extend(list(ngrams(sentence, 2,pad_left=True, pad_right=True, left_pad_symbol = SOS, right_pad_symbol = EOS)))
  trigram.extend(list(ngrams(sentence, 3, pad_left=True, pad_right=True, left_pad_symbol = SOS, right_pad_symbol = EOS)))

Now let's see how frequent our tokens are:

In [ ]:
unigram = remove_stopwords(unigram)
bigram = remove_stopwords(bigram)
trigram = remove_stopwords(trigram)
  
# generate frequency of n-grams 
freq_bi = FreqDist(bigram)
freq_tri = FreqDist(trigram)

Now, let's try and generate some sentences from our tri-gram language model

In [ ]:
d = {}
for a, b, c in freq_tri:
    if(a != None and b!= None and c!= None):
        if (a,b) not in d.keys():
            d[a,b] = {}
        if c not in d[a,b].keys():
            d[a,b][c] = freq_tri[a,b,c]
        else:
            d[a,b][c] += freq_tri[a,b,c]
# Next word prediction    
def pick_word(counter):
    "Chooses a random element."
    return random.choice(list(counter.items()))
s = [SOS,SOS]
num_tokens = 20
for i in range(num_tokens):
    if s[-2] == EOS and s[-1] == EOS:
        break
    prefix = (s[-2], s[-1])
    suffix = pick_word(d[prefix])
    s.append(suffix[0])
print(s)

As we can see, we have some output. In some cases it is fluent, but in most cases it isn't. Why do we think that is? Do you notice anything interesting about the type of sentences that the model is generating?

## Let's step it up a bit: Let's use NLTK

NLTK fortunately has a lot of this functionality baked into it. Let's trying using their classes and methods on different data.

In [ ]:
sentence_data = webtext.sents()
tokenized_text=[]
for sentence in sentence_data:
  sentence = list(map(lambda x:x.lower(),sentence))
  for word in sentence:
        if word== '.':
            sentence.remove(word) 
  tokenized_text.append(sentence)
tokenized_text[5]

Let's generate our sentences and our data for our tri-gram model

In [ ]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

Now let's fit our model

In [ ]:
from nltk.lm import MLE
model = MLE(n) # Lets train a 3-grams model, previously we set n=3

In [ ]:
model.fit(train_data, padded_sents)
model.vocab

In [ ]:
model.vocab.lookup(tokenized_text[6])

In [ ]:
model.vocab.lookup(["we", "want", "chatgpt", "to", "do", "my", "homework"])

In [ ]:
print(model.generate(20, random_seed=400))

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [ ]:
generate_sent(model, 20, random_seed=40)

## But wait there is more!

Did you notice something strange when we were creating our language model? We imported something called MLE – that stands for maximum likelihood estimation. This is a method of creating our language model where we estimate our probabilities directly from the data. Furthermore, in our case, we set our vocab cuttoff to 1: we wanted to ignore words that didn't occur frequently enough to provide us useful information. But what happens if we want to interpret the probability of a sentence that has such a word that we do not know in our vocabulary? Should the probability immediately be 0?

I think not. This problem is known as smoothing n-gram models! 

To find examples of smoothing, look at [these objects from `nltk.lm.models`](https://github.com/nltk/nltk/blob/develop/nltk/lm/models.py):

 - `Lidstone`: Provides Lidstone-smoothed scores.
 - `Laplace`: Implements Laplace (add one) smoothing.
 - `InterpolatedLanguageModel`: Logic common to all interpolated language models (Chen & Goodman 1995).
 - `WittenBellInterpolated`: Interpolated version of Witten-Bell smoothing.

In [ ]:
### TODO - IMPLEMENT YOUR OWN SMOOTHED N-GRAM MODELS!